In [19]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point

In [20]:
# import scripts for api calculation
import sys
import os

# Add the scripts folder to the Python path
scripts_path = os.path.abspath('../../scripts')

if scripts_path not in sys.path:
    sys.path.append(scripts_path)

from api_function import parse_coordinate

In [21]:
# Path to domain data and LGA
LGA_shapefile_path = "../../data/raw/ABS_LGA/LGA_2024_AUST_GDA94.shp"
crime = pd.read_csv("../../data/manual/CI_LGAa_data.csv")
property_data_path = "../../data/raw/individual_property.csv"

In [22]:
print(crime.head())
print(crime.info())

# Check specific columns for unexpected NaN values
print("Missing values per column:\n", crime.isna().sum())

        Police Region Local Government Area Display year ending  Year  \
0  1 North West Metro                 Total   Year ending March  2024   
1  1 North West Metro                 Yarra   Year ending March  2024   
2  1 North West Metro               Wyndham   Year ending March  2024   
3  1 North West Metro            Whittlesea   Year ending March  2024   
4  1 North West Metro             Nillumbik   Year ending March  2024   

  Incidents Recorded  
0            148,126  
1             10,482  
2             14,447  
3             10,112  
4              1,269  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 870 entries, 0 to 869
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Police Region          870 non-null    object
 1   Local Government Area  870 non-null    object
 2   Display year ending    870 non-null    object
 3   Year                   870 non-null    int64 
 4   Incident

In [23]:
crime = crime[crime['Year'] == 2023]

print(crime.head())
print(crime.info())

# Check specific columns for unexpected NaN values
print("Missing values per column:\n", crime.isna().sum())

         Police Region Local Government Area Display year ending  Year  \
15  1 North West Metro                 Total   Year ending March  2023   
16  1 North West Metro                 Yarra   Year ending March  2023   
17  1 North West Metro               Wyndham   Year ending March  2023   
18  1 North West Metro            Whittlesea   Year ending March  2023   
19  1 North West Metro             Nillumbik   Year ending March  2023   

   Incidents Recorded  
15            134,605  
16              9,533  
17             13,792  
18              9,164  
19              1,165  
<class 'pandas.core.frame.DataFrame'>
Index: 87 entries, 15 to 853
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Police Region          87 non-null     object
 1   Local Government Area  87 non-null     object
 2   Display year ending    87 non-null     object
 3   Year                   87 non-null     int64 
 4   I

In [24]:
# Load in datasets
LGA_gdf = gpd.read_file(LGA_shapefile_path)
property_data = pd.read_csv(property_data_path)

In [25]:
property_data

,Address,Cost,Bedrooms,Bathrooms,Coordinates,Closest Gov Secondary School,Gov Secondary Distance,Age under 20,Age 20-39,Age 40-59,...,2023,2026,CBD Distance,Train Distance,Electricity Distance,Hospital Distance,Library Distance,Park Distance,Tourist Attraction Distance,Grocery Distance
0,"901/22-40 Wills Street, Melbourne VIC 3000",$600 per week,1.0,1.0,"[-37.8107551, 144.9570001]",University High School,1.5 km away,8%,77%,12%,...,21566.0,24536.663914,0.9271,0.6166,0.8477,1.6536,1.5615,0.3633,1.3194,0.4771
1,"1207/270 King Street, Melbourne VIC 3000",$720 per week,2.0,2.0,"[-37.8136918, 144.9548583]",University High School,1.9 km away,6%,83%,10%,...,20027.0,23543.514398,0.9381,0.5685,1.3123,1.7040,1.7147,1.0145,0.8704,0.4587
2,"5809/442 ELIZABETH STREET, Melbourne VIC 3000",$850 Per Week ( Fully Furnished),2.0,1.0,"[-37.8084101, 144.9607759]",University High School,1.3 km away,3%,90%,7%,...,21566.0,24536.663914,0.6115,1.4196,0.8981,1.2886,1.1995,0.4621,0.9574,0.1615
3,"2112/80 A'beckett Street, Melbourne VIC 3000",$700 per week,2.0,2.0,"[-37.8089991, 144.9610792]",University High School,1.4 km away,3%,90%,7%,...,21566.0,24536.663914,0.5422,1.3504,0.8288,1.2193,1.1303,0.3928,0.8882,0.0923
4,"1210/81 A'beckett Street, Melbourne VIC 3000",$650 weekly,2.0,1.0,"[-37.8092536, 144.961181]",University High School,1.4 km away,6%,79%,12%,...,21566.0,24536.663914,0.5421,1.3502,0.8287,1.2192,1.1301,0.3927,0.8881,0.0921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7997,"79 Watt Street, Wonthaggi VIC 3995",$1950 PCM including GST,1.0,1.0,"[-38.6060046, 145.5910694]",Bass Coast College,0.7 km away,24%,25%,25%,...,28254.0,29211.295488,135.5041,105.7865,4.3976,1.1230,0.4730,0.2400,0.2684,0.1732
7998,"120-127 Mc Kenzie Street, Wonthaggi VIC 3995",$77 per calendar month,1.0,1.0,"[-38.5973378, 145.5930626]",Bass Coast College,1.4 km away,21%,16%,23%,...,28254.0,29211.295488,134.3962,104.6786,5.6409,2.3663,1.6573,0.3165,1.2473,1.0681
7999,"35 Anderson Avenue, Inverloch VIC 3996",$450.00 P/W,3.0,1.0,"[-38.6297675, 145.7392692]",Bass Coast College,12.6 km away,21%,14%,24%,...,28254.0,29211.295488,149.1981,119.4805,18.4882,15.2136,1.7612,0.8301,14.4135,14.2343
8000,"3 Kennards Court, Inverloch VIC 3996",$435,3.0,1.0,"[-38.6267337, 145.7286462]",Bass Coast College,11.6 km away,24%,16%,27%,...,28254.0,29211.295488,147.8189,118.1013,17.1090,13.8344,1.2459,0.8158,13.0343,12.8551


In [26]:
LGA_gdf

,LGA_CODE24,LGA_NAME24,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM,LOCI_URI21,geometry
0,10050,Albury,1,New South Wales,AUS,Australia,305.6386,https://linked.data.gov.au/dataset/asgsed3/LGA...,"POLYGON ((146.86565 -36.07293, 146.86511 -36.0..."
1,10180,Armidale,1,New South Wales,AUS,Australia,7809.4406,https://linked.data.gov.au/dataset/asgsed3/LGA...,"POLYGON ((152.38816 -30.52641, 152.38812 -30.5..."
2,10250,Ballina,1,New South Wales,AUS,Australia,484.9692,https://linked.data.gov.au/dataset/asgsed3/LGA...,"MULTIPOLYGON (((153.57106 -28.87382, 153.57105..."
3,10300,Balranald,1,New South Wales,AUS,Australia,21690.7493,https://linked.data.gov.au/dataset/asgsed3/LGA...,"POLYGON ((143.00432 -33.78165, 143.01538 -33.7..."
4,10470,Bathurst,1,New South Wales,AUS,Australia,3817.8645,https://linked.data.gov.au/dataset/asgsed3/LGA...,"POLYGON ((149.84877 -33.52785, 149.84863 -33.5..."
...,...,...,...,...,...,...,...,...,...
561,89799,Migratory - Offshore - Shipping (ACT),8,Australian Capital Territory,AUS,Australia,NaN,https://linked.data.gov.au/dataset/asgsed3/LGA...,None
562,99399,Unincorp. Other Territories,9,Other Territories,AUS,Australia,105.8806,https://linked.data.gov.au/dataset/asgsed3/LGA...,"MULTIPOLYGON (((167.94747 -29.12758, 167.94748..."
563,99499,No usual address (OT),9,Other Territories,AUS,Australia,NaN,https://linked.data.gov.au/dataset/asgsed3/LGA...,None
564,99799,Migratory - Offshore - Shipping (OT),9,Other Territories,AUS,Australia,NaN,https://linked.data.gov.au/dataset/asgsed3/LGA...,None


In [27]:
# Modify property to prepare join with shapefile
property_data[['latitude', 'longitude']] = property_data['Coordinates'].apply(lambda x: pd.Series(parse_coordinate(x)))
property_data['geometry'] = property_data.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
property_gdf = gpd.GeoDataFrame(property_data, geometry='geometry', crs=LGA_gdf.crs)

In [28]:
property_gdf

,Address,Cost,Bedrooms,Bathrooms,Coordinates,Closest Gov Secondary School,Gov Secondary Distance,Age under 20,Age 20-39,Age 40-59,...,2026,CBD Distance,Train Distance,Electricity Distance,Hospital Distance,Library Distance,Park Distance,Tourist Attraction Distance,Grocery Distance,geometry
0,"901/22-40 Wills Street, Melbourne VIC 3000",$600 per week,1.0,1.0,"[-37.8107551, 144.9570001]",University High School,1.5 km away,8%,77%,12%,...,24536.663914,0.9271,0.6166,0.8477,1.6536,1.5615,0.3633,1.3194,0.4771,POINT (144.95700 -37.81076)
1,"1207/270 King Street, Melbourne VIC 3000",$720 per week,2.0,2.0,"[-37.8136918, 144.9548583]",University High School,1.9 km away,6%,83%,10%,...,23543.514398,0.9381,0.5685,1.3123,1.7040,1.7147,1.0145,0.8704,0.4587,POINT (144.95486 -37.81369)
2,"5809/442 ELIZABETH STREET, Melbourne VIC 3000",$850 Per Week ( Fully Furnished),2.0,1.0,"[-37.8084101, 144.9607759]",University High School,1.3 km away,3%,90%,7%,...,24536.663914,0.6115,1.4196,0.8981,1.2886,1.1995,0.4621,0.9574,0.1615,POINT (144.96078 -37.80841)
3,"2112/80 A'beckett Street, Melbourne VIC 3000",$700 per week,2.0,2.0,"[-37.8089991, 144.9610792]",University High School,1.4 km away,3%,90%,7%,...,24536.663914,0.5422,1.3504,0.8288,1.2193,1.1303,0.3928,0.8882,0.0923,POINT (144.96108 -37.80900)
4,"1210/81 A'beckett Street, Melbourne VIC 3000",$650 weekly,2.0,1.0,"[-37.8092536, 144.961181]",University High School,1.4 km away,6%,79%,12%,...,24536.663914,0.5421,1.3502,0.8287,1.2192,1.1301,0.3927,0.8881,0.0921,POINT (144.96118 -37.80925)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7997,"79 Watt Street, Wonthaggi VIC 3995",$1950 PCM including GST,1.0,1.0,"[-38.6060046, 145.5910694]",Bass Coast College,0.7 km away,24%,25%,25%,...,29211.295488,135.5041,105.7865,4.3976,1.1230,0.4730,0.2400,0.2684,0.1732,POINT (145.59107 -38.60600)
7998,"120-127 Mc Kenzie Street, Wonthaggi VIC 3995",$77 per calendar month,1.0,1.0,"[-38.5973378, 145.5930626]",Bass Coast College,1.4 km away,21%,16%,23%,...,29211.295488,134.3962,104.6786,5.6409,2.3663,1.6573,0.3165,1.2473,1.0681,POINT (145.59306 -38.59734)
7999,"35 Anderson Avenue, Inverloch VIC 3996",$450.00 P/W,3.0,1.0,"[-38.6297675, 145.7392692]",Bass Coast College,12.6 km away,21%,14%,24%,...,29211.295488,149.1981,119.4805,18.4882,15.2136,1.7612,0.8301,14.4135,14.2343,POINT (145.73927 -38.62977)
8000,"3 Kennards Court, Inverloch VIC 3996",$435,3.0,1.0,"[-38.6267337, 145.7286462]",Bass Coast College,11.6 km away,24%,16%,27%,...,29211.295488,147.8189,118.1013,17.1090,13.8344,1.2459,0.8158,13.0343,12.8551,POINT (145.72865 -38.62673)


In [29]:
# Join
mapped_properties = gpd.sjoin(property_gdf, LGA_gdf, how='left', predicate='within')
mapped_properties['LGA_NAME24'] = mapped_properties['LGA_NAME24'].astype(str)

print(mapped_properties.head())

                                         Address  \
0     901/22-40 Wills Street, Melbourne VIC 3000   
1       1207/270 King Street, Melbourne VIC 3000   
2  5809/442 ELIZABETH STREET, Melbourne VIC 3000   
3   2112/80 A'beckett Street, Melbourne VIC 3000   
4   1210/81 A'beckett Street, Melbourne VIC 3000   

                               Cost  Bedrooms  Bathrooms  \
0                     $600 per week       1.0        1.0   
1                     $720 per week       2.0        2.0   
2  $850 Per Week ( Fully Furnished)       2.0        1.0   
3                     $700 per week       2.0        2.0   
4                       $650 weekly       2.0        1.0   

                  Coordinates Closest Gov Secondary School  \
0  [-37.8107551, 144.9570001]       University High School   
1  [-37.8136918, 144.9548583]       University High School   
2  [-37.8084101, 144.9607759]       University High School   
3  [-37.8089991, 144.9610792]       University High School   
4   [-37.8092536

In [30]:
crime['Local Government Area'] = crime['Local Government Area'].astype(str)

In [31]:
crime['Local Government Area'] = crime['Local Government Area'].str.strip()
mapped_properties['LGA_NAME24'] = mapped_properties['LGA_NAME24'].str.strip()


In [32]:
# Merge the spatially joined data with property and crime data using the correct column names
merged_data = mapped_properties.merge(
    crime, 
    left_on='LGA_NAME24', 
    right_on='Local Government Area', 
    how='left'
)

In [33]:
merged_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 8002 entries, 0 to 8001
Data columns (total 41 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   Address                       8002 non-null   object  
 1   Cost                          8002 non-null   object  
 2   Bedrooms                      7979 non-null   float64 
 3   Bathrooms                     7979 non-null   float64 
 4   Coordinates                   8002 non-null   object  
 5   Closest Gov Secondary School  6907 non-null   object  
 6   Gov Secondary Distance        6907 non-null   object  
 7   Age under 20                  7802 non-null   object  
 8   Age 20-39                     7802 non-null   object  
 9   Age 40-59                     7802 non-null   object  
 10  Age 60+                       7802 non-null   object  
 11  Postcode                      8002 non-null   int64   
 12  latitude                      8002 non-n

In [34]:
merged_data.head(3)

,Address,Cost,Bedrooms,Bathrooms,Coordinates,Closest Gov Secondary School,Gov Secondary Distance,Age under 20,Age 20-39,Age 40-59,...,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM,LOCI_URI21,Police Region,Local Government Area,Display year ending,Year,Incidents Recorded
0,"901/22-40 Wills Street, Melbourne VIC 3000",$600 per week,1.0,1.0,"[-37.8107551, 144.9570001]",University High School,1.5 km away,8%,77%,12%,...,Victoria,AUS,Australia,37.5452,https://linked.data.gov.au/dataset/asgsed3/LGA...,1 North West Metro,Melbourne,Year ending March,2023.0,"25,019"
1,"1207/270 King Street, Melbourne VIC 3000",$720 per week,2.0,2.0,"[-37.8136918, 144.9548583]",University High School,1.9 km away,6%,83%,10%,...,Victoria,AUS,Australia,37.5452,https://linked.data.gov.au/dataset/asgsed3/LGA...,1 North West Metro,Melbourne,Year ending March,2023.0,"25,019"
2,"5809/442 ELIZABETH STREET, Melbourne VIC 3000",$850 Per Week ( Fully Furnished),2.0,1.0,"[-37.8084101, 144.9607759]",University High School,1.3 km away,3%,90%,7%,...,Victoria,AUS,Australia,37.5452,https://linked.data.gov.au/dataset/asgsed3/LGA...,1 North West Metro,Melbourne,Year ending March,2023.0,"25,019"


In [35]:
selected_columns = [
    'Address', 'Cost', 'Bedrooms', 'Bathrooms', 'Coordinates', 
    'Closest Gov Secondary School', 'Gov Secondary Distance',
    'Age under 20', 'Age 20-39', 'Age 40-59', 'Age 60+', 'Postcode',
    'latitude', 'longitude', 'SA2  code', 'income_2020', '2023', '2026',
    'CBD Distance', 'Train Distance', 'Electricity Distance', 
    'Hospital Distance', 'Library Distance', 'Park Distance', 
    'Tourist Attraction Distance', 'Grocery Distance', 
    'Local Government Area', 'Incidents Recorded'
]

# Filter the merged data to keep only the selected columns
filtered_data = merged_data[selected_columns].copy()

In [36]:
filtered_data.head(3)

,Address,Cost,Bedrooms,Bathrooms,Coordinates,Closest Gov Secondary School,Gov Secondary Distance,Age under 20,Age 20-39,Age 40-59,...,CBD Distance,Train Distance,Electricity Distance,Hospital Distance,Library Distance,Park Distance,Tourist Attraction Distance,Grocery Distance,Local Government Area,Incidents Recorded
0,"901/22-40 Wills Street, Melbourne VIC 3000",$600 per week,1.0,1.0,"[-37.8107551, 144.9570001]",University High School,1.5 km away,8%,77%,12%,...,0.9271,0.6166,0.8477,1.6536,1.5615,0.3633,1.3194,0.4771,Melbourne,"25,019"
1,"1207/270 King Street, Melbourne VIC 3000",$720 per week,2.0,2.0,"[-37.8136918, 144.9548583]",University High School,1.9 km away,6%,83%,10%,...,0.9381,0.5685,1.3123,1.7040,1.7147,1.0145,0.8704,0.4587,Melbourne,"25,019"
2,"5809/442 ELIZABETH STREET, Melbourne VIC 3000",$850 Per Week ( Fully Furnished),2.0,1.0,"[-37.8084101, 144.9607759]",University High School,1.3 km away,3%,90%,7%,...,0.6115,1.4196,0.8981,1.2886,1.1995,0.4621,0.9574,0.1615,Melbourne,"25,019"


In [37]:
# Save the updated DataFrame if needed
filtered_data.to_csv("../../data/raw/individual_property_with_crime2023.csv", index=False)